In [15]:
import matplotlib.pyplot as plt
from Objects.Simulation import Simulation
import numpy as np
import pandas as pd
%pylab
%matplotlib inline



Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [16]:
%pylab
%matplotlib inline

from matplotlib import lines, markers
from cycler import cycler

# Create cycler object. Use any styling from above you please
monochromeWithMarker = (cycler('color', ['k']) * cycler('linestyle', ['-', '--', ':', '-.']) * cycler('marker', [ '.' ,'^']))
monochromeNoMarker = (cycler('color', ['k']) * cycler('linestyle', ['-', '--', ':', '-.']) * cycler('marker', [',']))
monochrome = monochromeNoMarker.concat(monochromeWithMarker)

plt.set_cmap('gray')
# Overriding styles for current script
plt.rcParams['axes.grid'] = True
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.spines.bottom'] = False
plt.rcParams['axes.spines.left'] = False
plt.rcParams['figure.figsize'] = (12,5)
mpl.rcParams['figure.dpi']= 300
plt.rcParams['axes.prop_cycle'] = monochrome

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


<Figure size 432x288 with 0 Axes>

In [17]:
def cal_one_p(p):
    return np.mean(p.delays), np.max(p.delays)

def cal_vdr(ps, tg):
    if type(ps) is not list:
        ps = [ps]
    total_vehPass = 0
    total_tg = 0
    for p in ps:
        total_vehPass += p.vehPass
        total_tg += tg
    return total_vehPass / total_tg * 3600

def get_data_array(p,tg):
    delaymean, delaymax = cal_one_p(p)
    return [delaymean, delaymax, cal_vdr(p, tg)]

In [19]:
simulation_time = 60 # seconds
n = 100 # number of vehicles
intended_speed = 80 * 1000/3600 # intended speed / speed limit of the road
sim = Simulation(time = simulation_time, avStep = 100)


LRT = [1.34, -1.34]
HSR = [0.58, -0.54]

human_profile = [1.7, -2.0]
av_profile = HSR

paras={
    'v_intend': intended_speed,
    'human_max_acc': human_profile[0],
    'human_max_dec': human_profile[1],
    'max_acc': av_profile[0],
    'max_dec': av_profile[1],
    'tsys': 0.8,
    'human_T': 1.8,
    'AV_T': 1.2
}

human = sim.run_human_simulation(n, paras)
get_data_array(human, simulation_time)

[0.09277343332193655, 0.23363514456212436, 1800.0]

In [5]:
human_ps = []
av_ps = []
cav_ps = []
start_acc = 1.34
end_acc = 0.58
start_dec = -1.34
end_dec = -0.54
totalLoop = int((start_acc - end_acc-0.0001) / 0.04 + 1) + 1
secondLoop = int((end_dec - start_dec -0.0001) / 0.04 + 1) + 1
print(totalLoop, secondLoop)
xs_acc = [round(x,2) for x in [start_acc - i * 0.04 for i in range(totalLoop)]]
ys_dec = [round(x,2) for x in [start_dec + j * 0.04 for j in range(secondLoop)]]

20 21


In [6]:
av_avg_delay = pd.DataFrame(index = ys_dec, columns=xs_acc)
av_max_delay = pd.DataFrame(index = ys_dec, columns=xs_acc)
av_vdr = pd.DataFrame(index = ys_dec, columns=xs_acc)
cav_avg_delay = pd.DataFrame(index = ys_dec, columns=xs_acc)
cav_max_delay = pd.DataFrame(index = ys_dec, columns=xs_acc)
cav_vdr = pd.DataFrame(index = ys_dec, columns=xs_acc)


for col in xs_acc:
    cur_acc = col
    for idx in ys_dec:
        cur_dec = idx
        paras['max_acc'] = cur_acc
        paras['max_dec'] = cur_dec
        print(cur_acc,cur_dec)
        av = sim.run_av_simulation(n, paras)
        cav = sim.run_cav_simulation(n, paras)
        av=get_data_array(av, simulation_time)
        cav=get_data_array(cav, simulation_time)
        print(cav)
        av_avg_delay.at[idx, col] = av[0]
        av_max_delay.at[idx, col] = av[1]
        av_vdr.at[idx, col] = av[2]
        cav_avg_delay.at[idx, col] = cav[0]
        cav_max_delay.at[idx, col] = cav[1]
        cav_vdr.at[idx, col] = cav[2]

1.34 -1.34
[0.04101435465221864, 0.09135513404961788, 3780.0]
1.34 -1.3
[0.04101427723857163, 0.09135442456566589, 3780.0]
1.34 -1.26
[0.041014195293642725, 0.09135367952364241, 3780.0]
1.34 -1.22
[0.04101410850596689, 0.09135289573072601, 3780.0]
1.34 -1.18
[0.04101401474193011, 0.09135206380384789, 3780.0]
1.34 -1.14
[0.041013915302314796, 0.09135118784821211, 3780.0]
1.34 -1.1
[0.041013808561788885, 0.09135025370450932, 3780.0]
1.34 -1.06
[0.04101369356171434, 0.09134926010297227, 3780.0]
1.34 -1.02
[0.041013570000904165, 0.09134820464961631, 3780.0]
1.34 -0.98
[0.04101343719026153, 0.09134707665632934, 3780.0]
1.34 -0.94
[0.04101329254678841, 0.09134586420471265, 3780.0]
1.34 -0.9
[0.04101313695917007, 0.09134456975468853, 3780.0]
1.34 -0.86
[0.041012967134062474, 0.09134317018640557, 3780.0]
1.34 -0.82
[0.04101278219529584, 0.09134166049116887, 3780.0]
1.34 -0.78
[0.041012580718136536, 0.09134002716281045, 3780.0]
1.34 -0.74
[0.04101235970068275, 0.09133825563796351, 3780.0]
1.34 

[0.043629186017881974, 0.09514954977704938, 3720.0000000000005]
1.14 -0.78
[0.043629036592790896, 0.0951482828407821, 3720.0000000000005]
1.14 -0.74
[0.043628869675895784, 0.09514688939102303, 3720.0000000000005]
1.14 -0.7
[0.043628686947939484, 0.09514536988352572, 3720.0000000000005]
1.14 -0.66
[0.043628483162825096, 0.0951436980107859, 3720.0000000000005]
1.14 -0.62
[0.043628254883734556, 0.09514184522879034, 3720.0000000000005]
1.14 -0.58
[0.04362800223070951, 0.09513981338599849, 3720.0000000000005]
1.14 -0.54
[0.043627714988979056, 0.09513753705281901, 3720.0000000000005]
1.1 -1.34
[0.04364492931618154, 0.09436896993970464, 3660.0]
1.1 -1.3
[0.04364488474484041, 0.09436852101982439, 3660.0]
1.1 -1.26
[0.04364483675077878, 0.09436804687945134, 3660.0]
1.1 -1.22
[0.04364478660760848, 0.09436755780402918, 3660.0]
1.1 -1.18
[0.043644732068776004, 0.09436703767502612, 3660.0]
1.1 -1.14
[0.04364467383617093, 0.09436648557105207, 3660.0]
1.1 -1.1
[0.04364461142969175, 0.0943658998801375

[0.047581791481778095, 0.09939014012851183, 3540.0]
0.9 -0.54
[0.04758163202288472, 0.09938877403212663, 3540.0]
0.86 -1.34
[0.04910682814695672, 0.1019502464949358, 3540.0]
0.86 -1.3
[0.04910680507164627, 0.10194997019847056, 3540.0]
0.86 -1.26
[0.049106780154197255, 0.10194967966305309, 3540.0]
0.86 -1.22
[0.04910675349951042, 0.10194937573780186, 3540.0]
0.86 -1.18
[0.04910672494515916, 0.1019490564771025, 3540.0]
0.86 -1.14
[0.04910669370472924, 0.10194871928910247, 3540.0]
0.86 -1.1
[0.04910666045153586, 0.10194836465168243, 3540.0]
0.86 -1.06
[0.04910662337641306, 0.10194798118454311, 3540.0]
0.86 -1.02
[0.049106583647955095, 0.10194757502808233, 3540.0]
0.86 -0.98
[0.04910653950923801, 0.10194713683088465, 3540.0]
0.86 -0.94
[0.04910649121944222, 0.10194666759557403, 3540.0]
0.86 -0.9
[0.0491064381265949, 0.1019461589239763, 3540.0]
0.86 -0.86
[0.049106379357555734, 0.10194560655783255, 3540.0]
0.86 -0.82
[0.0491063142303083, 0.1019450036786506, 3540.0]
0.86 -0.78
[0.04910624172

[0.058651311756203556, 0.11248671068198414, 3240.0]
0.62 -1.06
[0.05865130435479003, 0.11248660270668402, 3240.0]
0.62 -1.02
[0.058651295943380095, 0.11248648429545371, 3240.0]
0.62 -0.98
[0.05865128624526722, 0.11248635198468666, 3240.0]
0.62 -0.94
[0.05865127499056526, 0.11248620404210162, 3240.0]
0.62 -0.9
[0.05865126205652991, 0.11248604013535865, 3240.0]
0.62 -0.86
[0.05865124699411889, 0.1124858558298902, 3240.0]
0.62 -0.82
[0.05865122944620539, 0.11248564794584709, 3240.0]
0.62 -0.78
[0.05865120891077308, 0.11248541305543522, 3240.0]
0.62 -0.74
[0.05865118544991041, 0.11248515224903854, 3240.0]
0.62 -0.7
[0.05865115708533323, 0.11248484887457065, 3240.0]
0.62 -0.66
[0.058651124447081586, 0.11248450908725623, 3240.0]
0.62 -0.62
[0.05865108566910166, 0.11248411792588389, 3240.0]
0.62 -0.58
[0.05865103965727321, 0.11248366786262189, 3240.0]
0.62 -0.54
[0.058650985591385395, 0.11248315287575422, 3240.0]
0.58 -1.34
[0.0614727531871404, 0.11573706671586757, 3180.0]
0.58 -1.3
[0.061472

In [9]:
av_avg_delay * 1000

,1.34,1.3,1.26,1.22,1.18,1.14,1.1,1.06,1.02,0.98,0.94,0.9,0.86,0.82,0.78,0.74,0.7,0.66,0.62,0.58
-1.34,74.3855,75.9433,77.615,79.4087,79.1068,81.1574,83.3611,83.4366,85.9764,88.7112,89.3133,92.4944,95.9286,97.2768,101.316,105.693,108.12,113.342,119.038,123.076
-1.30,76.0265,75.4493,77.1099,78.8934,78.5895,80.6307,82.8263,85.1874,85.4327,88.1626,91.1044,91.9416,95.3756,96.7237,100.766,105.15,107.58,112.815,118.527,122.575
-1.26,75.5166,74.9381,76.5867,78.359,80.2637,80.0839,82.2706,84.6243,84.8672,87.5915,90.5297,91.3655,94.7988,98.5119,100.193,104.583,107.017,112.264,117.992,122.051
-1.22,74.9888,76.5315,76.0444,77.8046,79.6983,79.516,81.6927,84.0381,84.2786,86.9964,89.9304,90.7645,94.1967,97.9115,99.593,103.989,106.428,111.686,117.431,121.501
-1.18,74.4419,75.971,75.4819,77.2289,79.1105,78.9257,81.0913,83.4274,83.6652,86.3757,89.3046,90.137,93.5673,97.2832,98.9657,103.368,108.156,111.081,116.843,120.925
-1.14,73.8751,75.3893,74.8983,76.6307,78.4989,80.5129,80.4649,82.7905,85.3009,85.7276,88.6504,89.4811,92.9086,96.6251,98.3084,102.716,107.514,110.446,116.225,120.318
-1.10,73.2874,74.7853,76.3999,76.0088,77.8623,79.8629,79.8119,82.1256,84.6263,85.0502,87.9659,91.1227,92.2185,95.9348,97.6189,102.032,106.84,109.778,115.574,119.68
-1.06,72.6776,74.1578,75.7555,75.3617,77.199,79.1848,79.1305,81.431,83.9206,84.3414,87.2487,90.4002,91.4945,95.2097,96.8946,101.312,106.129,109.074,114.888,119.007
-1.02,72.0448,73.5056,75.0848,76.7913,76.5074,78.4767,80.6074,80.7044,83.1814,85.8644,86.4964,89.6412,90.7339,94.447,98.4864,100.554,105.38,108.332,114.164,118.296
-0.98,73.3877,72.8274,74.3861,76.0732,75.7858,77.7367,79.8508,79.9438,82.4063,85.0775,85.7063,88.8429,89.9337,93.6435,97.6841,99.7542,104.589,107.548,113.397,117.544


In [13]:
subFolder = "SG1/"
av_avg_delay.to_csv(subFolder+'SG1_av_avg_delay.csv')
av_max_delay.to_csv(subFolder+'SG1_av_max_delay.csv')
av_vdr.to_csv(subFolder+'SG1_av_vdr.csv')
cav_avg_delay.to_csv(subFolder+'SG1_cav_avg_delay.csv')
cav_max_delay.to_csv(subFolder+'SG1_cav_max_delay.csv')
cav_vdr.to_csv(subFolder+'SG1_cav_vdr.csv')